In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Read the train and test datasets using pandas, already imported
train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
train.head()

In [ ]:
#First, check if there are NaN in the train or test data
train[train.isna().any(axis = 1)].head()
test[test.isna().any(axis = 1)].head()


In [ ]:
#There are NaN values, so use an strategy to fill them: Imputation
#However, first separate the validation group to avoid contamination during imputation
X = train.copy()
y = X.SalePrice
X.drop(['SalePrice'], axis=1, inplace=True)
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(
                                        X, y, train_size=0.8, test_size=0.2, random_state=0)

In [ ]:
from sklearn.impute import SimpleImputer

my_imputer = SimpleImputer(strategy='most_frequent')
imputed_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_valid = pd.DataFrame(my_imputer.transform(X_valid))

#Make sure the columns and indexes stay congruent
imputed_train.columns = X_train.columns
imputed_valid.columns = X_valid.columns

imputed_train.index = X_train.index
imputed_valid.index = X_valid.index

# Take care of categorical variables
# Get list of categorical variables
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index) #Selects the name of columns with previous condition true

#Handle_unknown: When this parameter is set to ‘ignore’ and an unknown category is 
#encountered during transform, the resulting one-hot encoded columns for this feature 
#will be all zeros.
#Sparse: Will return sparse matrix if set True else will return an array.
from sklearn.preprocessing import OneHotEncoder
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(imputed_train[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(imputed_valid[object_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = imputed_train.index
OH_cols_valid.index = imputed_valid.index

# Remove categorical columns (will replace with one-hot encoding)
num_train = imputed_train.drop(object_cols, axis=1)
num_valid = imputed_valid.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_train = pd.concat([num_train, OH_cols_train], axis=1)
OH_valid = pd.concat([num_valid, OH_cols_valid], axis=1)

#Imputation removed the right types, and the nerwork complains
OH_train = OH_train.astype('int64')
OH_valid = OH_valid.astype('int64')

In [ ]:
#Check your results
OH_train.head()
OH_valid.head()

In [ ]:
#Using neural networks
from tensorflow import keras
from tensorflow.keras import layers
from keras.optimizers import Adam

model = keras.Sequential([
    layers.BatchNormalization(input_dim=OH_train.shape[1]),
    layers.Dense(128, activation='sigmoid'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(16, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(1),
])

optimizer = Adam(lr=0.005, decay=0.)
'''MSLE will treat small differences between small true and predicted values 
approximately the same as big differences between large true and predicted values
Use MSLE when doing regression, believing that your target, conditioned on the 
input, is normally distributed, and you don’t want large errors to be 
significantly more penalized than small ones, in those cases where the range 
of the target value is large.'''
model.compile(optimizer=optimizer,
             loss='msle', 
             metrics=['mse'])

from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=10, # how many epochs to wait before stopping
    restore_best_weights=True,
)

In [ ]:
history = model.fit(
    OH_train, y_train,
    validation_data=(OH_valid, y_valid),
    batch_size=128,
    epochs=10000,
    verbose=0,
    callbacks=[early_stopping]
)

In [ ]:
# Get predictions
imputed_test = pd.DataFrame(my_imputer.transform(test))
imputed_test.columns = test.columns
imputed_test.index = test.index

OH_cols_test = pd.DataFrame(OH_encoder.transform(imputed_test[object_cols]))

# One-hot encoding removed index; put it back
OH_cols_test.index = imputed_test.index

# Remove categorical columns (will replace with one-hot encoding)
num_test = imputed_test.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_test = pd.concat([num_test, OH_cols_test], axis=1)

#Imputation removed the right types, and the nerwork complains
OH_test = OH_test.astype('int64')

sale_price_preds = model.predict(OH_test)
print(sale_price_preds)

output = pd.DataFrame({'Id': OH_test.Id,
                       'SalePrice': sale_price_preds.T[0]})
output.to_csv('/kaggle/working/submit.csv', index=False)
